In [ ]:
# Configure shared library path
# This allows the notebook to use packages installed in the shared directory
import sys

SITE_SHARED = "/workspace/site-packages_shared"

if SITE_SHARED not in sys.path:
    sys.path.append(SITE_SHARED)
    
print(f"✓ Shared library path configured: {SITE_SHARED}")


## Landsat Demonstration Notebook for Snowflake Testing

This notebook demonstrates Landsat data extraction. The baseline data is [Landsat Collection 2 Level 2](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) data from the MS Planetary Computer catalog. 

First, we need to enable External Access in this workbook to support API and PyPI connections.

Click the menu in top-right corner of this page and select Notebook Settings.

Select "DATA_CHALLENGE_EXTERNAL_ACCESS"

If you do not see this option, and you already created the External Access by following the steps here (__link to Solution Center__), refresh this browser tab.


In [ ]:
!pip install -r requirements.txt

### Load Python Dependencies

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os

<h3>Extracting Landsat Data Using API Calls</h3> <p align="justify"> The API-based method allows us to efficiently access <b>Landsat</b> data for specific coordinates and time periods, ensuring scalability and reproducibility of the process. </p> <p align="justify"> Through the API, we can query individual bands or compute indices like <b>NDMI</b> on-the-fly. This approach reduces storage requirements and simplifies data preprocessing, making it ideal for large-scale environmental and water quality analysis. </p>

<p>The <b>compute_Landsat_values</b> function extracts Landsat surface reflectance values for specific sampling locations using a 100 m focal buffer around each point. For each location:</p>

<ul>
  <li>A bounding box (bbox) is created around the latitude and longitude coordinates.</li>
  <li>The Microsoft Planetary Computer API is queried for Landsat-8 Level-2 surface reflectance imagery within the date range.</li>
  <li>The nearest low-cloud (<10% cloud cover) scene is selected, and the specified bands (<b>green</b>, <b>nir08</b>, <b>swir16</b>, <b>swir22</b>) are loaded.</li>
  <li>Median values of the pixels within the bounding box are computed to reduce the effect of noise or outliers.</li>
</ul>

<p><b>Why the buffer value is 0.00089831:</b></p>

<p>We want a ~100 m buffer around each point. At the equator, 1 degree ≈ 110 km. Therefore, the degree equivalent of 100 m is:</p>

<p style="text-align:center;">
  <em>buffer_deg = 100 m / 110,000 m/deg ≈ 0.00089831</em>
</p>

<p>This slightly adjusted value ensures that the buffer approximately matches the pixel resolution of Landsat imagery, capturing a ~100 m area around each sampling location.</p>


In [2]:
# Setup
tqdm.pandas()

def compute_Landsat_values(row):
    lat = row['Latitude']
    lon = row['Longitude']
    date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')

    # Buffer size for ~100m 
    bbox_size = 0.00089831  
    bbox = [
        lon - bbox_size / 2,
        lat - bbox_size / 2,
        lon + bbox_size / 2,
        lat + bbox_size / 2
    ]

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    # Wider search range, we'll filter to nearest date later
    search = catalog.search(
        collections=["landsat-c2-l2"],
        bbox=bbox,
        datetime="2011-01-01/2015-12-31",
        query={"eo:cloud_cover": {"lt": 10}},
    )
    
    items = search.item_collection()

    if not items:
        return pd.Series({
            "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
        })

    try:
        # Convert sample date to UTC
        sample_date_utc = date.tz_localize("UTC") if date.tzinfo is None else date.tz_convert("UTC")

        # Pick the item closest to the sample date
        items = sorted(
            items,
            key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc)
        )
        selected_item = pc.sign(items[0])

        # Load required bands
        bands_of_interest = ["green", "nir08", "swir16", "swir22"]
        data = stac_load([selected_item], bands=bands_of_interest, bbox=bbox).isel(time=0)

        green = data["green"].astype("float")
        nir = data["nir08"].astype("float")
        swir16 = data["swir16"].astype("float")
        swir22 = data["swir22"].astype("float")
        
        # Compute medians
        median_green = float(green.median(skipna=True).values)
        median_nir = float(nir.median(skipna=True).values)
        median_swir16 = float(swir16.median(skipna=True).values)
        median_swir22 = float(swir22.median(skipna=True).values)

        # Replace 0 with NaN
        median_green = median_green if median_green != 0 else np.nan
        median_nir = median_nir if median_nir != 0 else np.nan
        median_swir16 = median_swir16 if median_swir16 != 0 else np.nan
        median_swir22 = median_swir22 if median_swir22 != 0 else np.nan
        
        return pd.Series({
            "nir": median_nir,
            "green": median_green,
            "swir16": median_swir16,
            "swir22": median_swir22,
        })
    
    except Exception as e:
        return pd.Series({
            "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
        })


### Extracting features for the training dataset

In [3]:
Water_Quality_df=pd.read_csv('water_quality_training_dataset_100.csv')
Water_Quality_df.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,1/2/11,128.912,555.0,10
1,-26.861111,28.884722,1/3/11,74.720,162.9,163
2,-26.450000,28.085833,1/3/11,89.254,573.0,80
3,-27.671111,27.236944,1/3/11,82.000,203.6,101
4,-27.356667,27.286389,1/3/11,56.100,145.1,151


In [4]:
Water_Quality_df.shape

(100, 6)

In [5]:
# Extract band values from Landsat for training dataset
train_features_path = "landsat_features_output_SF.csv"

print("🚀 Running Landsat feature extraction for training data...")
landsat_train_features = Water_Quality_df.progress_apply(compute_Landsat_values, axis=1)
landsat_train_features.to_csv(train_features_path, index=False)

🚀 Running Landsat feature extraction for training data...


100%|██████████| 100/100 [07:08<00:00,  4.29s/it]


<p><b>NDMI and MNDWI Indices:</b></p>
<p>In this notebook, we compute two commonly used water-related indices from the extracted Landsat bands:</p>
<ul>
  <li><b>NDMI (Normalized Difference Moisture Index):</b> Measures vegetation water content and surface moisture. Computed as <em>(NIR - SWIR16) / (NIR + SWIR16)</em>.</li>
  <li><b>MNDWI (Modified Normalized Difference Water Index):</b> Highlights open water features by enhancing water reflectance and suppressing built-up areas. Computed as <em>(Green - SWIR16) / (Green + SWIR16)</em>.</li>
</ul>

<p>An <b>epsilon value</b> (<em>eps = 1e-10</em>) is added in the denominators to avoid division by zero. These indices are widely used in hydrological and water quality analyses for detecting water presence and vegetation moisture levels.</p>


In [6]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_train_features['NDMI'] = (landsat_train_features['nir'] - landsat_train_features['swir16']) / (landsat_train_features['nir'] + landsat_train_features['swir16'] + eps)
landsat_train_features['MNDWI'] = (landsat_train_features['green'] - landsat_train_features['swir16']) / (landsat_train_features['green'] + landsat_train_features['swir16'] + eps)

In [7]:
landsat_train_features['Latitude'] = Water_Quality_df['Latitude']
landsat_train_features['Longitude'] = Water_Quality_df['Longitude']
landsat_train_features['Sample Date'] = Water_Quality_df['Sample Date']
landsat_train_features = landsat_train_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [8]:
landsat_train_features.to_csv(train_features_path, index=False)

In [9]:
# Preview File
landsat_train_features.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI
0,-28.760833,17.730278,1/2/11,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595
1,-26.861111,28.884722,1/3/11,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134
2,-26.450000,28.085833,1/3/11,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805
3,-27.671111,27.236944,1/3/11,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416
4,-27.356667,27.286389,1/3/11,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683
